<a href="https://colab.research.google.com/github/alec-carawan/Intro-to-Machine-Learning/blob/main/HW5_Q2_%26_Q3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [146]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import torch.optim as optim
import torch

In [147]:
housingSet   =   pd.read_csv('https://raw.githubusercontent.com/alec-carawan/datasets/main/Housing.csv')

In [148]:
# List of variables to map

varlist =  ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']

# Defining the map function
def binary_map(x):
    return x.map({'yes': 1, 'no': 0})

# Applying the function to the housing list
housingSet[varlist] = housingSet[varlist].apply(binary_map)

#map furnishing status
housingSet['furnishingstatus'] = housingSet['furnishingstatus'].map({'furnished': 2, 'semi-furnished': 1, 'unfurnished': 0})

In [149]:
#STANDARDIZE DATA

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

num_vars = ['price','area','bedrooms','bathrooms','stories','mainroad','guestroom','basement','hotwaterheating','airconditioning','parking','prefarea','furnishingstatus']
#using num_vars keeps data types as data frames, instead of np arrays

housingSet[num_vars] = scaler.fit_transform(housingSet[num_vars])

In [150]:
features = ['area','bedrooms','bathrooms','stories','parking']
# Separating out the features
X = housingSet.loc[:, features].values
# Separating out the target
y = housingSet.loc[:,['price']].values

In [151]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, test_size = 0.2, )

Linear model:

w1 * area + w2 * bedrooms + w3 * bathrooms + w4 * stories + w5 * parking + b

In [152]:
def model(area, bedrooms, bathrooms, stories, parking, w1, w2, w3, w4, w5, b):
    return w1 * area + w2 * bedrooms + w3 * bathrooms + w4 * stories + w5 * parking + b

In [153]:
def loss_fn(pred, targ):
    squared_diffs = (pred - targ)**2
    return squared_diffs.mean()

In [154]:
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
learning_rate = 1e-5
optimizer = optim.SGD([params], lr=learning_rate)

area_t       = torch.tensor(X_train[:, 0])
bedrooms_t   = torch.tensor(X_train[:, 1])
bathrooms_t  = torch.tensor(X_train[:, 2])
stories_t    = torch.tensor(X_train[:, 3])
parking_t    = torch.tensor(X_train[:, 4])
price_t      = torch.tensor(y_train[:, :])

area_v       = torch.tensor(X_test[:, 0])
bedrooms_v   = torch.tensor(X_test[:, 1])
bathrooms_v  = torch.tensor(X_test[:, 2])
stories_v    = torch.tensor(X_test[:, 3])
parking_v    = torch.tensor(X_test[:, 4])
price_v      = torch.tensor(y_test[:, :])

In [155]:
pred = model(area_t, bedrooms_t, bathrooms_t, stories_t, parking_t, *params)
loss = loss_fn(pred, price_t)
loss.backward()

optimizer.step()

params

tensor([ 9.9999e-01,  9.9999e-01,  1.0000e+00,  9.9999e-01,  9.9999e-01,
        -1.9027e-05], requires_grad=True)

In [156]:
def training_loop(n_epochs, optimizer, params,
                  area_t, bedrooms_t, bathrooms_t, stories_t, parking_t, price_t,
                  area_v, bedrooms_v, bathrooms_v, stories_v, parking_v, price_v):
    for epoch in range(1, n_epochs + 1):
        train_pred = model(area_t, bedrooms_t, bathrooms_t, stories_t, parking_t, *params)
        train_loss = loss_fn(train_pred, price_t)

        val_pred = model(area_v, bedrooms_v, bathrooms_v, stories_v, parking_v, *params)
        val_loss = loss_fn(val_pred, price_v)

        optimizer.zero_grad()
        train_loss.backward() # <2>
        optimizer.step()

        if epoch <= 3 or epoch % 500 == 0:
            print(f"Epoch {epoch}, Training loss {train_loss.item():.4f},"
                  f" Validation loss {val_loss.item():.4f}")

    return params


In [157]:
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
learning_rate = 1e-2
optimizer = optim.SGD([params], lr=learning_rate)

training_loop(
    n_epochs = 5000, optimizer = optimizer, params = params,
    area_t = area_t, bedrooms_t = bedrooms_t, bathrooms_t = bathrooms_t, stories_t = stories_t, parking_t = parking_t, price_t = price_t,
    area_v = area_v, bedrooms_v = bedrooms_v, bathrooms_v = bathrooms_v, stories_v = stories_v, parking_v = parking_v, price_v = price_v
    )

Epoch 1, Training loss 1.3349, Validation loss 1.5305
Epoch 2, Training loss 1.2785, Validation loss 1.4676
Epoch 3, Training loss 1.2251, Validation loss 1.4080
Epoch 500, Training loss 0.0931, Validation loss 0.0947
Epoch 1000, Training loss 0.0467, Validation loss 0.0480
Epoch 1500, Training loss 0.0345, Validation loss 0.0369
Epoch 2000, Training loss 0.0304, Validation loss 0.0334
Epoch 2500, Training loss 0.0286, Validation loss 0.0318
Epoch 3000, Training loss 0.0275, Validation loss 0.0308
Epoch 3500, Training loss 0.0269, Validation loss 0.0302
Epoch 4000, Training loss 0.0264, Validation loss 0.0298
Epoch 4500, Training loss 0.0262, Validation loss 0.0295
Epoch 5000, Training loss 0.0260, Validation loss 0.0293


tensor([ 0.0937,  0.1092, -0.0026, -0.0290, -0.0317,  0.2045],
       requires_grad=True)

In [158]:
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
learning_rate = 1e-2
optimizer = optim.Adam([params], lr=learning_rate)

training_loop(
    n_epochs = 5000, optimizer = optimizer, params = params,
    area_t = area_t, bedrooms_t = bedrooms_t, bathrooms_t = bathrooms_t, stories_t = stories_t, parking_t = parking_t, price_t = price_t,
    area_v = area_v, bedrooms_v = bedrooms_v, bathrooms_v = bathrooms_v, stories_v = stories_v, parking_v = parking_v, price_v = price_v
    )

Epoch 1, Training loss 1.3349, Validation loss 1.5305
Epoch 2, Training loss 1.2853, Validation loss 1.4745
Epoch 3, Training loss 1.2369, Validation loss 1.4197
Epoch 500, Training loss 0.0290, Validation loss 0.0322
Epoch 1000, Training loss 0.0258, Validation loss 0.0291
Epoch 1500, Training loss 0.0256, Validation loss 0.0288
Epoch 2000, Training loss 0.0256, Validation loss 0.0288
Epoch 2500, Training loss 0.0256, Validation loss 0.0288
Epoch 3000, Training loss 0.0256, Validation loss 0.0288
Epoch 3500, Training loss 0.0256, Validation loss 0.0288
Epoch 4000, Training loss 0.0256, Validation loss 0.0288
Epoch 4500, Training loss 0.0256, Validation loss 0.0288
Epoch 5000, Training loss 0.0256, Validation loss 0.0288


tensor([ 9.7383e-08,  2.3006e-07, -4.8021e-08,  3.1744e-11,  3.2183e-09,
         2.5553e-01], requires_grad=True)

In [159]:
features = ['area','bedrooms','bathrooms','stories', 'mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'parking', 'prefarea', 'furnishingstatus']
# Separating out the features
X = housingSet.loc[:, features].values
# Separating out the target
y = housingSet.loc[:,['price']].values

In [160]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, test_size = 0.2, )

In [161]:
def model(area, bedrooms, bathrooms, stories, mainroad, guestroom, basement, hotwaterheating, airconditioning, parking, prefarea, furnishingstatus, w1, w2, w3, w4, w5, w6, w7, w8, w9, w10, w11, w12, b):
    return w1 * area + w2 * bedrooms + w3 * bathrooms + w4 * stories + w5 * mainroad + w6 * guestroom + w7 * basement + w8 * hotwaterheating + w9 * airconditioning + w10 * parking + w11 * prefarea + w12 * furnishingstatus + b

In [162]:
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
learning_rate = 1e-4
optimizer = optim.SGD([params], lr=learning_rate)

area_t                = torch.tensor(X_train[:, 0])
bedrooms_t            = torch.tensor(X_train[:, 1])
bathrooms_t           = torch.tensor(X_train[:, 2])
stories_t             = torch.tensor(X_train[:, 3])
mainroad_t            = torch.tensor(X_train[:, 4])
guestroom_t           = torch.tensor(X_train[:, 5])
basement_t            = torch.tensor(X_train[:, 6])
hotwaterheating_t     = torch.tensor(X_train[:, 7])
airconditioning_t     = torch.tensor(X_train[:, 8])
parking_t             = torch.tensor(X_train[:, 9])
prefarea_t            = torch.tensor(X_train[:, 10])
furnishingstatus_t    = torch.tensor(X_train[:, 11])
price_t               = torch.tensor(y_train[:, :])

area_v                = torch.tensor(X_test[:, 0])
bedrooms_v            = torch.tensor(X_test[:, 1])
bathrooms_v           = torch.tensor(X_test[:, 2])
stories_v             = torch.tensor(X_test[:, 3])
mainroad_v            = torch.tensor(X_test[:, 4])
guestroom_v           = torch.tensor(X_test[:, 5])
basement_v            = torch.tensor(X_test[:, 6])
hotwaterheating_v     = torch.tensor(X_test[:, 7])
airconditioning_v     = torch.tensor(X_test[:, 8])
parking_v             = torch.tensor(X_test[:, 9])
prefarea_v            = torch.tensor(X_test[:, 10])
furnishingstatus_v    = torch.tensor(X_test[:, 11])
price_v               = torch.tensor(y_test[:, :])

In [163]:
pred = model(area_t, bedrooms_t, bathrooms_t, stories_t, mainroad_t, guestroom_t, basement_t, hotwaterheating_t, airconditioning_t, parking_t, prefarea_t, furnishingstatus_t, *params)
loss = loss_fn(pred, price_t)
loss.backward()

optimizer.step()

params

tensor([ 9.9982e-01,  9.9972e-01,  9.9991e-01,  9.9979e-01,  9.9937e-01,
         9.9982e-01,  9.9969e-01,  9.9996e-01,  9.9971e-01,  9.9980e-01,
         9.9977e-01,  9.9963e-01, -6.7955e-04], requires_grad=True)

In [164]:
def training_loop(n_epochs, optimizer, params,
                  area_t, bedrooms_t, bathrooms_t, stories_t, mainroad_t, guestroom_t, basement_t, hotwaterheating_t, airconditioning_t, parking_t, prefarea_t, furnishingstatus_t, price_t,
                  area_v, bedrooms_v, bathrooms_v, stories_v, mainroad_v, guestroom_v, basement_v, hotwaterheating_v, airconditioning_v, parking_v, prefarea_v, furnishingstatus_v, price_v):
    for epoch in range(1, n_epochs + 1):
        train_pred = model(area_t, bedrooms_t, bathrooms_t, stories_t, mainroad_t, guestroom_t, basement_t, hotwaterheating_t, airconditioning_t, parking_t, prefarea_t, furnishingstatus_t, *params)
        train_loss = loss_fn(train_pred, price_t)

        val_pred = model(area_v, bedrooms_v, bathrooms_v, stories_v, mainroad_v, guestroom_v, basement_v, hotwaterheating_v, airconditioning_v, parking_v, prefarea_v, furnishingstatus_v, *params)
        val_loss = loss_fn(val_pred, price_v)

        optimizer.zero_grad()
        train_loss.backward() # <2>
        optimizer.step()

        if epoch <= 3 or epoch % 500 == 0:
            print(f"Epoch {epoch}, Training loss {train_loss.item():.4f},"
                  f" Validation loss {val_loss.item():.4f}")

    return params

In [165]:
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
learning_rate = 1e-2
optimizer = optim.SGD([params], lr=learning_rate)

training_loop(
    n_epochs = 5000, optimizer = optimizer, params = params,
    area_t = area_t, bedrooms_t = bedrooms_t, bathrooms_t = bathrooms_t, stories_t = stories_t, mainroad_t = mainroad_t, guestroom_t = guestroom_t, basement_t = basement_t, hotwaterheating_t = hotwaterheating_t, airconditioning_t = airconditioning_t, parking_t = parking_t, prefarea_t = prefarea_t, furnishingstatus_t = furnishingstatus_t, price_t = price_t,
    area_v = area_v, bedrooms_v = bedrooms_v, bathrooms_v = bathrooms_v, stories_v = stories_v, mainroad_v = mainroad_v, guestroom_v = guestroom_v, basement_v = basement_v, hotwaterheating_v = hotwaterheating_v, airconditioning_v = airconditioning_v, parking_v = parking_v, prefarea_v = prefarea_v, furnishingstatus_v = furnishingstatus_v, price_v = price_v
    )

Epoch 1, Training loss 14.1932, Validation loss 15.5565
Epoch 2, Training loss 12.7619, Validation loss 14.0363
Epoch 3, Training loss 11.4825, Validation loss 12.6753
Epoch 500, Training loss 0.1098, Validation loss 0.1343
Epoch 1000, Training loss 0.0533, Validation loss 0.0633
Epoch 1500, Training loss 0.0377, Validation loss 0.0434
Epoch 2000, Training loss 0.0317, Validation loss 0.0362
Epoch 2500, Training loss 0.0290, Validation loss 0.0331
Epoch 3000, Training loss 0.0277, Validation loss 0.0316
Epoch 3500, Training loss 0.0269, Validation loss 0.0308
Epoch 4000, Training loss 0.0264, Validation loss 0.0304
Epoch 4500, Training loss 0.0261, Validation loss 0.0301
Epoch 5000, Training loss 0.0258, Validation loss 0.0299


tensor([ 0.0988,  0.1188, -0.0031, -0.0325,  0.0098, -0.0005, -0.0043,  0.0145,
        -0.0020, -0.0306, -0.0063, -0.0005,  0.2013], requires_grad=True)

In [167]:
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
learning_rate = 1e-2
optimizer = optim.Adam([params], lr=learning_rate)

training_loop(
    n_epochs = 5000, optimizer = optimizer, params = params,
    area_t = area_t, bedrooms_t = bedrooms_t, bathrooms_t = bathrooms_t, stories_t = stories_t, mainroad_t = mainroad_t, guestroom_t = guestroom_t, basement_t = basement_t, hotwaterheating_t = hotwaterheating_t, airconditioning_t = airconditioning_t, parking_t = parking_t, prefarea_t = prefarea_t, furnishingstatus_t = furnishingstatus_t, price_t = price_t,
    area_v = area_v, bedrooms_v = bedrooms_v, bathrooms_v = bathrooms_v, stories_v = stories_v, mainroad_v = mainroad_v, guestroom_v = guestroom_v, basement_v = basement_v, hotwaterheating_v = hotwaterheating_v, airconditioning_v = airconditioning_v, parking_v = parking_v, prefarea_v = prefarea_v, furnishingstatus_v = furnishingstatus_v, price_v = price_v
    )

Epoch 1, Training loss 14.1932, Validation loss 15.5565
Epoch 2, Training loss 10.7463, Validation loss 11.8286
Epoch 3, Training loss 7.8025, Validation loss 8.6376
Epoch 500, Training loss 0.0254, Validation loss 0.0294
Epoch 1000, Training loss 0.0254, Validation loss 0.0294
Epoch 1500, Training loss 0.0254, Validation loss 0.0294
Epoch 2000, Training loss 0.0254, Validation loss 0.0294
Epoch 2500, Training loss 0.0254, Validation loss 0.0294
Epoch 3000, Training loss 0.0254, Validation loss 0.0294
Epoch 3500, Training loss 0.0254, Validation loss 0.0294
Epoch 4000, Training loss 0.0254, Validation loss 0.0294
Epoch 4500, Training loss 0.0254, Validation loss 0.0294
Epoch 5000, Training loss 0.0254, Validation loss 0.0294


tensor([ 1.0025e-08,  4.3838e-08, -8.9852e-09, -2.3951e-09,  1.5930e-08,
        -3.0055e-10,  2.0372e-09,  1.1295e-09,  2.8229e-10, -2.1855e-09,
        -1.9814e-09,  3.7969e-09,  2.6057e-01], requires_grad=True)